In [1]:
!pip install tabpfn
!pip install tabpfn_extensions


[notice] A new release of pip is available: 24.1.2 -> 25.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from tabpfn import TabPFNClassifier
from tabpfn_extensions.many_class import ManyClassClassifier

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score



In [4]:
trains = ['train100', 'train200', 'train300', 'train478']
tests = ['test100', 'test200', 'test300', 'test478', ]
traindir = '../../../../data/top30groups/traindata'
testdir = '../../../../data/top30groups/testdata'

trainpaths = []
testpaths = []
for i, partition in enumerate(trains):
    trainpaths.append(f'{traindir}/{partition}.csv')
    testpaths.append(f'{testdir}/{tests[i]}.csv')

In [5]:
def preprocess_data(trainpath, testpath):
    traindata = pd.read_csv(trainpath, encoding='ISO-8859-1')
    testdata = pd.read_csv(testpath, encoding='ISO-8859-1')

    y_train_raw = traindata['gname']
    y_test_raw = testdata['gname']

    train_features = traindata.drop(columns='gname')
    test_features = testdata.drop(columns=['gname'])

    # One-hot encoding for geodata
    geodata = ['longitude', 'latitude']
    all_categories = pd.concat([train_features, test_features])
    onehot = pd.get_dummies(all_categories, columns=geodata)

    train_features = onehot.iloc[:len(train_features)]
    test_features = onehot.iloc[len(train_features):]

    le = LabelEncoder()
    y_train = le.fit_transform(y_train_raw)
    y_test = le.transform(y_test_raw)

    X_train = train_features
    X_test = test_features

    return X_train, y_train, X_test, y_test, le

In [6]:
models = []
accuracies = []
predictions = []
truths = []
rocs = []
n = [100, 200, 300, 478]
for i, path in enumerate(trainpaths):
    X_train, y_train, X_test, y_test, le = preprocess_data(trainpaths[i], testpaths[i])
    
    print(f'Computing for partition {i+1}, balanced classes with {n[i]} entries for each class and one hot encoded geodata')

    # run classifier on data
    clf = TabPFNClassifier(ignore_pretraining_limits=True)
    many_class_clf = ManyClassClassifier(
        estimator=clf,
        alphabet_size=10
    )
    many_class_clf.fit(X_train, y_train)

    prediction = many_class_clf.predict(X_test)
    accuracy = accuracy_score(y_test, prediction)
    print("Accuracy", accuracy)

    decoded_preds = le.inverse_transform(prediction)
    decoded_truths = le.inverse_transform(y_test)
    
    #  saving models, accuracies, predictions, and truths in case we need it for statistics later
    models.append(clf)
    accuracies.append(accuracy)
    predictions.append(decoded_preds)
    truths.append(decoded_truths)

    print("---------------------------------------------------")

Computing for partition 1, balanced classes with 100 entries for each class and one hot encoded geodata
Using codebook with 24 estimators and 30 classes


AttributeError: 'ManyClassClassifier' object has no attribute 'estimators_'

In [ ]:
for i in range(len(models)):
    print(classification_report(predictions[i], truths[i]))